ChatGPT

In [ ]:
import numpy as np

def findEssentialMatrix8pt(pts1_8pt, pts2_8pt, K):
    # Нормализуем точки, преобразовывая их из пиксельных в нормализованные координаты камеры
    pts1_normalized = (np.linalg.inv(K) @ np.hstack((pts1_8pt, np.ones((pts1_8pt.shape[0], 1)))).T).T
    pts2_normalized = (np.linalg.inv(K) @ np.hstack((pts2_8pt, np.ones((pts2_8pt.shape[0], 1)))).T).T
    
    # Составляем матрицу A для уравнений эпиполярного ограничения
    A = np.zeros((8, 9))
    for i in range(8):
        x1, y1, _ = pts1_normalized[i]
        x2, y2, _ = pts2_normalized[i]
        A[i] = [x1 * x2, x1 * y2, x1, y1 * x2, y1 * y2, y1, x2, y2, 1]

    # Используем SVD для нахождения фундаментальной матрицы (последняя строка V^T)
    _, _, V = np.linalg.svd(A)
    F = V[-1].reshape(3, 3)

    # Принуждаем фундаментальную матрицу быть ранга 2
    U, S, Vt = np.linalg.svd(F)
    S = [1, 1, 0]  # Сингулярные числа для существенной матрицы
    E = U @ np.diag(S) @ Vt

    return E

def findEssentialMatrixRansac(pts1, pts2, K, samples=2000, threshold=0.001):
    max_inliers = 0
    best_E = None
    best_inliers_mask = None

    for _ in range(samples):
        # Выбираем случайные 8 точек
        indices = np.random.choice(len(pts1), 8, replace=False)
        pts1_8pt = pts1[indices]
        pts2_8pt = pts2[indices]

        # Находим E для выбранных точек
        E = findEssentialMatrix8pt(pts1_8pt, pts2_8pt, K)

        # Проверяем количество инлайеров
        inliers_mask = []
        inliers_count = 0
        for i in range(len(pts1)):
            x1 = np.array([pts1[i][0], pts1[i][1], 1.0])
            x2 = np.array([pts2[i][0], pts2[i][1], 1.0])

            # Эпиполярное ограничение x2.T * E * x1
            error = abs(x2.T @ E @ x1)

            if error < threshold:
                inliers_mask.append(True)
                inliers_count += 1
            else:
                inliers_mask.append(False)

        # Обновляем наилучшую модель, если текущее количество инлайеров больше
        if inliers_count > max_inliers:
            max_inliers = inliers_count
            best_E = E
            best_inliers_mask = inliers_mask

    return best_E, np.array(best_inliers_mask, dtype=bool)


Claude

In [ ]:
import numpy as np

def findEssentialMatrix8pt(pts1_8pt, pts2_8pt, K):
    """
    8-point algorithm for Essential Matrix estimation
    Args:
        pts1_8pt: (Nx2) points in first image
        pts2_8pt: (Nx2) points in second image
        K: (3x3) camera intrinsic matrix
    Returns:
        E: (3x3) Essential matrix
    """
    # Normalize points using K
    pts1_norm = np.linalg.inv(K) @ np.vstack((pts1_8pt.T, np.ones(pts1_8pt.shape[0])))
    pts2_norm = np.linalg.inv(K) @ np.vstack((pts2_8pt.T, np.ones(pts2_8pt.shape[0])))
    
    pts1_norm = pts1_norm[:2].T
    pts2_norm = pts2_norm[:2].T

    # Construct A matrix
    A = np.zeros((pts1_8pt.shape[0], 9))
    for i in range(pts1_8pt.shape[0]):
        x1, y1 = pts1_norm[i]
        x2, y2 = pts2_norm[i]
        A[i] = [x1*x2, x1*y2, x1, y1*x2, y1*y2, y1, x2, y2, 1]

    # Solve Ax = 0 using SVD
    _, _, V = np.linalg.svd(A)
    E_flat = V[-1]
    E = E_flat.reshape(3, 3)

    # Enforce Essential matrix properties
    U, _, Vt = np.linalg.svd(E)
    E = U @ np.diag([1, 1, 0]) @ Vt

    return E

def findEssentialMatrixRansac(pts1, pts2, K, samples=2000, threshold=0.001):
    """
    RANSAC algorithm for robust Essential Matrix estimation
    Args:
        pts1: (Nx2) points in first image
        pts2: (Nx2) points in second image
        K: (3x3) camera intrinsic matrix
        samples: number of RANSAC iterations
        threshold: threshold for inlier classification
    Returns:
        best_E: (3x3) best Essential matrix
        best_mask: (Nx1) boolean mask indicating inliers
    """
    best_num_inliers = 0
    best_E = None
    best_mask = None
    N = pts1.shape[0]

    # Normalize points using K
    pts1_norm = np.linalg.inv(K) @ np.vstack((pts1.T, np.ones(N)))
    pts2_norm = np.linalg.inv(K) @ np.vstack((pts2.T, np.ones(N)))
    
    pts1_norm = pts1_norm[:2].T
    pts2_norm = pts2_norm[:2].T

    for i in range(samples):
        # Randomly select 8 points
        idx = np.random.choice(N, 8, replace=False)
        E = findEssentialMatrix8pt(pts1[idx], pts2[idx], K)

        # Calculate epipolar constraint for all points
        pts1_h = np.hstack((pts1_norm, np.ones((N, 1))))
        pts2_h = np.hstack((pts2_norm, np.ones((N, 1))))
        
        # Calculate epipolar lines
        lines = pts2_h @ E.T
        # Calculate point-to-line distances
        distances = np.abs(np.sum(lines * pts1_h, axis=1)) / np.sqrt(lines[:,0]**2 + lines[:,1]**2)
        
        # Create inlier mask
        mask = distances < threshold
        num_inliers = np.sum(mask)

        # Update best model if we found more inliers
        if num_inliers > best_num_inliers:
            best_num_inliers = num_inliers
            best_E = E
            best_mask = mask

    return best_E, best_mask

Claude

In [ ]:
def recoverPose(E, pts1, pts2, K):
    """
    Восстанавливает позу камеры из существенной матрицы E
    
    Args:
        E (ndarray): Существенная матрица 3x3
        pts1 (ndarray): Точки на первом изображении (Nx2)
        pts2 (ndarray): Точки на втором изображении (Nx2)
        K (ndarray): Калибровочная матрица камеры 3x3
    
    Returns:
        R (ndarray): Матрица поворота 3x3
        t (ndarray): Вектор переноса 3x1
    """
    # SVD разложение матрицы E
    U, _, Vt = np.linalg.svd(E)
    
    # Проверка определителя для обеспечения правой системы координат
    if np.linalg.det(U) < 0:
        U = -U
    if np.linalg.det(Vt) < 0:
        Vt = -Vt
    
    # Матрица W для получения R
    W = np.array([[0, -1, 0],
                  [1, 0, 0],
                  [0, 0, 1]])
    
    # 4 возможные гипотезы для R и t
    R1 = U @ W @ Vt
    R2 = U @ W.T @ Vt
    t1 = U[:, 2].reshape(3, 1)
    t2 = -t1
    
    poses = [(R1, t1), (R1, t2), (R2, t1), (R2, t2)]
    
    # Конвертация точек в однородные координаты
    pts1_norm = np.linalg.inv(K) @ np.vstack((pts1.T, np.ones(pts1.shape[0])))
    pts2_norm = np.linalg.inv(K) @ np.vstack((pts2.T, np.ones(pts2.shape[0])))
    
    max_points_front = -1
    best_pose = None
    
    # Проверка каждой гипотезы
    for R, t in poses:
        points_front = 0
        P1 = np.eye(3, 4)  # [I|0]
        P2 = np.hstack((R, t))  # [R|t]
        
        # Триангуляция точек для текущей гипотезы
        points_4d = triangulatePoints(pts1_norm[:2].T, pts2_norm[:2].T, R, t, np.eye(3))
        points_3d = points_4d[:, :3] / points_4d[:, 3:]
        
        # Проверка глубины для обеих камер
        depths1 = points_3d[:, 2]
        depths2 = (R @ points_3d.T + t).T[:, 2]
        
        # Подсчет точек перед обеими камерами
        points_front = np.sum((depths1 > 1) & (depths2 > 1))
        
        if points_front > max_points_front:
            max_points_front = points_front
            best_pose = (R, t)
    
    return best_pose[0], best_pose[1]

def triangulatePoints(pts1, pts2, R, t, K):
    """
    Триангулирует 3D точки из соответствующих точек на двух изображениях
    
    Args:
        pts1 (ndarray): Точки на первом изображении (Nx2)
        pts2 (ndarray): Точки на втором изображении (Nx2)
        R (ndarray): Матрица поворота 3x3
        t (ndarray): Вектор переноса 3x1
        K (ndarray): Калибровочная матрица камеры 3x3
    
    Returns:
        points_4d (ndarray): Триангулированные точки в однородных координатах (Nx4)
    """
    # Формируем матрицы проекции
    P1 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))  # [K|0]
    P2 = K @ np.hstack((R, t))  # [KR|Kt]
    
    points_4d = []
    
    for pt1, pt2 in zip(pts1, pts2):
        # Формируем матрицу A для DLT
        A = np.zeros((4, 4))
        
        # Для первой камеры
        A[0] = pt1[0] * P1[2] - P1[0]
        A[1] = pt1[1] * P1[2] - P1[1]
        
        # Для второй камеры
        A[2] = pt2[0] * P2[2] - P2[0]
        A[3] = pt2[1] * P2[2] - P2[1]
        
        # Решаем систему AX = 0
        _, _, Vt = np.linalg.svd(A)
        point_4d = Vt[-1]
        points_4d.append(point_4d)
    
    return np.array(points_4d)

ChatGPT

In [ ]:
import numpy as np

def recoverPose(E, pts1, pts2, K):
    # Разложение E на R и t с помощью SVD
    U, _, Vt = np.linalg.svd(E)
    
    # Принудительно корректируем возможное нарушение детерминанта
    if np.linalg.det(U) < 0:
        U[:, -1] *= -1
    if np.linalg.det(Vt) < 0:
        Vt[-1, :] *= -1

    # Кандидаты на R и t
    R_candidates = [U @ np.diag([1, 1, -1]) @ Vt,
                    U @ np.diag([1, 1, -1]) @ Vt]
    t_candidates = [U[:, 2], -U[:, 2]]

    # Выбираем лучшую комбинацию R и t
    max_positive_depth = 0
    best_R = None
    best_t = None

    for R in R_candidates:
        for t in t_candidates:
            # Триангулируем точки и проверяем количество с положительной глубиной
            depths = triangulatePoints(pts1, pts2, R, t, K)
            positive_depths = np.sum(depths > 1)

            # Обновляем лучшую комбинацию, если больше точек с положительной глубиной
            if positive_depths > max_positive_depth:
                max_positive_depth = positive_depths
                best_R = R
                best_t = t

    return best_R, best_t

def triangulatePoints(pts1, pts2, R, t, K):
    # Построение проекционных матриц
    P1 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))
    P2 = K @ np.hstack((R, t.reshape(-1, 1)))

    # Триангуляция для каждой пары точек
    depths = []
    for i in range(len(pts1)):
        # Построение уравнений для метода наименьших квадратов
        x1, y1 = pts1[i]
        x2, y2 = pts2[i]

        A = np.array([
            x1 * P1[2] - P1[0],
            y1 * P1[2] - P1[1],
            x2 * P2[2] - P2[0],
            y2 * P2[2] - P2[1]
        ])

        # Решение уравнений A * X = 0 с помощью SVD
        _, _, Vt = np.linalg.svd(A)
        X = Vt[-1]
        X /= X[-1]  # Приведение к однородным координатам

        # Глубина относительно второй камеры (третья координата)
        depth = (R[2] @ X[:3] + t[2])
        depths.append(depth)

    return np.array(depths)


In [ ]:
import numpy as np

def findEssentialMatrix8pt(pts1_8pt, pts2_8pt, K):
    """
    Находит существенную матрицу используя 8-точечный алгоритм
    
    Args:
        pts1_8pt (ndarray): Точки на первом изображении (8x2)
        pts2_8pt (ndarray): Точки на втором изображении (8x2)
        K (ndarray): Калибровочная матрица камеры (3x3)
    
    Returns:
        E (ndarray): Существенная матрица (3x3)
    """
    # Нормализация точек
    pts1_h = np.hstack((pts1_8pt, np.ones((8, 1)))).T
    pts2_h = np.hstack((pts2_8pt, np.ones((8, 1)))).T

    # Преобразование в нормализованные координаты
    pts1_norm = (np.linalg.inv(K) @ pts1_h).T[:, :2]  # Берем только x,y
    pts2_norm = (np.linalg.inv(K) @ pts2_h).T[:, :2]  # Берем только x,y

    # Нормализация для улучшения численной стабильности
    # Центрирование и масштабирование
    mean1 = np.mean(pts1_norm, axis=0)
    mean2 = np.mean(pts2_norm, axis=0)
    scale1 = np.sqrt(2) / np.std(pts1_norm - mean1)
    scale2 = np.sqrt(2) / np.std(pts2_norm - mean2)

    T1 = np.array([[scale1, 0, -scale1*mean1[0]],
                   [0, scale1, -scale1*mean1[1]],
                   [0, 0, 1]])
    
    T2 = np.array([[scale2, 0, -scale2*mean2[0]],
                   [0, scale2, -scale2*mean2[1]],
                   [0, 0, 1]])

    # Применяем нормализацию
    pts1_normalized = (T1 @ np.hstack((pts1_norm, np.ones((8, 1)))).T).T[:, :2]
    pts2_normalized = (T2 @ np.hstack((pts2_norm, np.ones((8, 1)))).T).T[:, :2]

    # Инициализация матрицы A
    A = np.zeros((8, 9))
    for i in range(8):
        x1, y1 = pts1_normalized[i]
        x2, y2 = pts2_normalized[i]
        
        A[i] = [x2*x1, x2*y1, x2, y2*x1, y2*y1, y2, x1, y1, 1]

    # Решение системы уравнений через SVD
    _, _, V = np.linalg.svd(A)
    F = V[-1].reshape(3, 3)

    # Применяем SVD к F и enforcing rank-2 constraint
    U, _, Vt = np.linalg.svd(F)
    
    # Денормализация
    F = T2.T @ F @ T1

    # Преобразование F в E используя калибровочную матрицу
    E = K.T @ F @ K

    # Enforcing правильные сингулярные значения для E
    U, _, Vt = np.linalg.svd(E)
    E = U @ np.diag([1, 1, 0]) @ Vt

    return E

ChatGPT

In [ ]:
import numpy as np

def normalize_points(points):
    """
    Нормализует точки для повышения численной устойчивости,
    используя центрирование и масштабирование.
    """
    centroid = np.mean(points, axis=0)
    shifted_points = points - centroid
    avg_dist = np.mean(np.sqrt(np.sum(shifted_points**2, axis=1)))
    scale = np.sqrt(2) / avg_dist
    
    T = np.array([[scale, 0, -scale * centroid[0]],
                  [0, scale, -scale * centroid[1]],
                  [0, 0, 1]])
    
    normalized_points = (T @ np.hstack((points, np.ones((points.shape[0], 1)))).T).T
    return normalized_points[:, :2], T

def findEssentialMatrix8pt(pts1_8pt, pts2_8pt, K):
    # Применяем нормализацию точек
    pts1_8pt_norm, T1 = normalize_points(pts1_8pt)
    pts2_8pt_norm, T2 = normalize_points(pts2_8pt)

    # Строим матрицу A
    A = np.zeros((8, 9))
    for i in range(8):
        x1, y1 = pts1_8pt_norm[i]
        x2, y2 = pts2_8pt_norm[i]
        A[i] = [x1 * x2, x1 * y2, x1, y1 * x2, y1 * y2, y1, x2, y2, 1]

    # Находим фундаментальную матрицу с помощью SVD
    _, _, V = np.linalg.svd(A)
    F_norm = V[-1].reshape(3, 3)

    # Применяем принудительное ранговое ограничение
    U, _, Vt = np.linalg.svd(F_norm)
    F_norm = U @ np.diag([1, 1, 0]) @ Vt

    # Денормализуем фундаментальную матрицу
    F = T2.T @ F_norm @ T1

    # Преобразуем фундаментальную матрицу в существенную с помощью матрицы K
    E = K.T @ F @ K

    # Приводим E к согласованному масштабу
    E /= np.linalg.norm(E)
    
    return E

def findEssentialMatrixRansac(pts1, pts2, K, samples=2000, threshold=0.001, seed=42):
    np.random.seed(seed)
    best_E = None
    best_mask = None
    max_inliers = 0
    N = pts1.shape[0]

    for _ in range(samples):
        # Выбор случайных 8 точек
        indices = np.random.choice(N, 8, replace=False)
        pts1_8pt = pts1[indices]
        pts2_8pt = pts2[indices]

        # Вычисляем E с использованием 8-ми точечного алгоритма
        E_candidate = findEssentialMatrix8pt(pts1_8pt, pts2_8pt, K)

        # Проверяем количество инлайеров
        mask_inliers = []
        num_inliers = 0
        for i in range(N):
            x1 = np.hstack((pts1[i], 1))
            x2 = np.hstack((pts2[i], 1))
            
            # Вычисляем эпиполярное ограничение
            error = abs(x2.T @ E_candidate @ x1)
            
            if error < threshold:
                mask_inliers.append(True)
                num_inliers += 1
            else:
                mask_inliers.append(False)

        # Обновляем лучшие значения при наличии большего числа инлайеров
        if num_inliers > max_inliers:
            max_inliers = num_inliers
            best_E = E_candidate
            best_mask = np.array(mask_inliers, dtype=bool)

    return best_E, best_mask
